In [5]:
import duckdb
import pandas as pd

pd.set_option('display.max_rows', None)
con = duckdb.connect()

con.execute("""
CREATE OR REPLACE TABLE combine_data AS
SELECT * FROM read_csv('C:\\Users\\RaymondCarpenter\\Documents\\GitHub\\nfl-draft-data\\combine_data_unique_athlete_id_step4.csv');
""")

con.execute("""
CREATE OR REPLACE TABLE filtered_player_stats AS
SELECT * FROM read_csv('C:\\Users\\RaymondCarpenter\\Documents\\GitHub\\nfl-draft-data\\filtered_player_stats_full.csv');
""")

mismatches = con.execute("""
SELECT athlete_id, player, POS_GP, 'Only in Combine Data' AS source
FROM combine_data
WHERE athlete_id NOT IN (SELECT athlete_id FROM filtered_player_stats)

UNION ALL

SELECT athlete_id, player, '' as POS_GP, 'Only in Filtered Player Stats' AS source
FROM filtered_player_stats
WHERE athlete_id NOT IN (SELECT athlete_id FROM combine_data);
""").fetchdf()

# Display mismatches
print(mismatches)


     athlete_id                   player POS_GP                source
0       5084939             Isaiah Adams     OL  Only in Combine Data
1       4883090          Kiran Amegadjie     OL  Only in Combine Data
2       5145719          Gottlieb Ayedze     OL  Only in Combine Data
3       4426638          Karsen Barnhart     OL  Only in Combine Data
4       4430957          Tanor Bortolini     OL  Only in Combine Data
5       4692037             Javion Cohen     OL  Only in Combine Data
6       4695883              Jalen Coker     WR  Only in Combine Data
7       4361691               Frank Crum     OL  Only in Combine Data
8       4579564             Anim Dankwah     OL  Only in Combine Data
9       4695404             Isaiah Davis     RB  Only in Combine Data
10      4366636              Willie Drew     DB  Only in Combine Data
11      4571627           Ethan Driskell     OL  Only in Combine Data
12      4567410         Kingsley Eguakun     OL  Only in Combine Data
13      4570883     

In [9]:
con.execute("""
CREATE OR REPLACE TABLE combine_data AS
SELECT * FROM read_csv('C:\\Users\\RaymondCarpenter\\Documents\\GitHub\\nfl-draft-data\\combine_data_unique_athlete_id_step4.csv');
""")

no_athlete_id = con.execute("""
SELECT athlete_id, player, POS_GP
FROM combine_data
WHERE athlete_id IS NULL;
""").fetchdf()

print(no_athlete_id)

      athlete_id                       player POS_GP
0            NaN                 TyRon Hopper     LB
1            NaN              JhaQuan Jackson     WR
2            NaN               Cedric Johnson     DL
3            NaN                     Matt Lee     OL
4            NaN                  KT Leveston     OL
5            NaN               Hunter Nourzad     OL
6            NaN             Eyabi Okie-Anoma     DL
7            NaN                 Patrick Paul     OL
8            NaN                 Prince Pines     OL
9            NaN               Keith Randolph     DL
10           NaN               Leonard Taylor     DL
11           NaN                Nathan Thomas     OL
12           NaN     Sedrick Van Pran-Granger     OL
13           NaN               Caedan Wallace     OL
14           NaN                 Nate Wiggins     DB
15           NaN               James Williams     DB
16           NaN                  Mekhi Wingo     DL
17           NaN                   Zak Zinter 

In [11]:
import duckdb

# Connect to an in-memory DuckDB database
con = duckdb.connect()

# Load CSV files into DuckDB tables
con.execute("""
CREATE OR REPLACE TABLE combine_2007 AS
SELECT * FROM read_csv('C:\\Users\\RaymondCarpenter\\Documents\\GitHub\\nfl-draft-data\\combine_2007.csv');
""")

con.execute("""
CREATE OR REPLACE TABLE filtered_player_stats AS
SELECT * FROM read_csv('C:\\Users\\RaymondCarpenter\\Documents\\GitHub\\nfl-draft-data\\filtered_player_stats_full.csv');
""")

# Attempt to fill missing athlete_ids by matching name and college
filled_athlete_ids = con.execute("""
SELECT 
    c.player, 
    c.college, 
    f.athlete_id
FROM combine_2007 c
LEFT JOIN filtered_player_stats f
    ON c.player = f.player AND c.college = f.team
WHERE c.athlete_id IS NULL AND f.athlete_id IS NOT NULL
""").fetchdf()

# Display results with matched athlete_ids
print(filled_athlete_ids)


                 Player                College  athlete_id
0            Aaron Ross                  Texas      120230
1          Dallas Baker                Florida      127394
2          Yamon Figurs           Kansas State      128819
3           Tony Taylor                Georgia      133501
4        Desmond Bishop             California      133537
5              Roy Hall             Ohio State      133632
6           John Stocco              Wisconsin      133680
7        Marcus Paschal                   Iowa      133737
8        Steve Breaston               Michigan      133761
9          David Harris               Michigan      133764
10       Brandon Fields         Michigan State      133782
11         Drew Stanton         Michigan State      133795
12         Matt Trannon         Michigan State      133797
13     Sabby Piscitelli           Oregon State      133983
14          Josh Gattis            Wake Forest      133998
15           Zac Taylor               Nebraska      1340

In [12]:
import duckdb
import pandas as pd

con = duckdb.connect()

con.execute("""
CREATE OR REPLACE TABLE combine_2007 AS
SELECT * FROM read_csv('C:\\Users\\RaymondCarpenter\\Documents\\GitHub\\nfl-draft-data\\combine_2007.csv');
""")

con.execute("""
CREATE OR REPLACE TABLE filtered_player_stats AS
SELECT * FROM read_csv('C:\\Users\\RaymondCarpenter\\Documents\\GitHub\\nfl-draft-data\\filtered_player_stats_full.csv');
""")

con.execute("""
UPDATE combine_2007
SET athlete_id = f.athlete_id
FROM filtered_player_stats AS f
WHERE combine_2007.player = f.player 
  AND combine_2007.college = f.team
  AND combine_2007.athlete_id IS NULL
  AND f.athlete_id IS NOT NULL
""")

updated_df = con.execute("SELECT * FROM combine_2007").fetchdf()
updated_df.to_csv('C:\\Users\\RaymondCarpenter\\Documents\\GitHub\\nfl-draft-data\\combine_2007_updated.csv', index=False)

print("CSV updated and saved as 'combine_2007_updated.csv'.")


CSV updated and saved as 'combine_2007_updated.csv'.


In [8]:
import duckdb
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
con = duckdb.connect(database=':memory:')

query = """
    SELECT
        player,
        (man_yards_after_catch_per_reception + zone_yards_after_catch_per_reception)/2 AS total_yac_per_rec
    FROM read_csv_auto('C:/Users/RaymondCarpenter/Documents/GitHub/nfl-draft-data/receiving_scheme.csv')
    WHERE base_targets >= 50 AND position = 'WR'
    ORDER BY total_yac_per_rec DESC
    LIMIT 100
"""
result_df = con.execute(query).fetchdf()

print(result_df)


                   player  total_yac_per_rec
0              Ryan Wingo              12.25
1    Roderick Daniels Jr.              11.20
2       Antwane Wells Jr.              10.75
3              Kenny Odom              10.15
4            Samuel Brown              10.05
5           Kyle Williams               9.65
6              Tre Harris               9.50
7          Eric McAlister               9.45
8        Malik Rutherford               8.80
9            DT Sheffield               8.65
10           Dante Wright               8.65
11       Zachariah Branch               8.65
12            Isaiah Bond               8.55
13         Jimmy Horn Jr.               8.55
14           Jalen Royals               8.50
15        Monaray Baldwin               8.50
16           Dalvin Smith               8.20
17            Arian Smith               8.15
18            Noah Thomas               8.15
19       Jamaal Pritchett               8.05
20       Vinny Anthony II               8.05
21        